## APIs project

Submit your code on github... even partially completed and get an invitation to a special kiwi pub night. Filip will send you an email with details.

### Project Requirements:

#### Part 1
Write a Flask server with the following routes:
  - `/vehicles`
    - returns json information for all vehicles
  - `/vehicles/plot`
    - returns a plot of the location of all vehicles
  - `/routes`
    - returns json information for all vehicles grouped by route
  - `/routes/plot`
    - returns a plot of the location of all vehicles colored by route including legend
  - `/routes/<int:id>`
    - returns json information for the vehicles on route `id`

#### Part 2
Calculate average vehicle distance by route over a time interval:
  - Request vehicle movement data once per minute for 30 minutes
  - For each vehicle
    - Calculate the absolute differences between it's current position and it's previous position (latitude / longitude) using the provided function
    - Sum the total distance traveled the vehicle over the 30 minute interval
    - The output of this step should be a single column with shape `(len(vehicles) x 1)`
  - Group the vehicles by route and calculate the average distance traveled by a vehicle on that route over the 30 minute interval
    - The input to this step should be the total vehicle distance you calculated in the last step
    - Plot a bar chart of the grouped results
    - Summarize results in a few sentences.

#### Bonus
- For each vehicle, plot a line tracing its positions over the 30 minute interval

## Getting Started

In [ ]:
%matplotlib inline
import matplotlib
import matplotlib.pyplot as plt
matplotlib.style.use('ggplot')

import pandas as pd
import requests
import time

In [ ]:
vehicle_url = 'http://ris.azurewebsites.net/vehicles.json'
r = requests.get(vehicle_url)
df = pd.DataFrame.from_dict(r.json())

### Notes
- Make sure you filter out vehicles with latitude and longitude of 0

In [ ]:
df_slice = df[(df.longitude != 0) & df.latitude != 0]
df_slice.plot.scatter(x='latitude', y='longitude');

### Calculating distance

In [ ]:
# http://stackoverflow.com/a/21623206
from math import cos, asin, sqrt
def distance(lat1, lon1, lat2, lon2):
    p = 0.017453292519943295
    a = 0.5 - cos((lat2 - lat1) * p)/2 + cos(lat1 * p) * cos(lat2 * p) * (1 - cos((lon2 - lon1) * p)) / 2
    return 12742 * asin(sqrt(a))

In [ ]:
df_slice.head()

In [ ]:
lat1 = df_slice.loc[0,'latitude']
lon1 = df_slice.loc[0,'longitude']
lat2 = df_slice.loc[1,'latitude']
lon2 = df_slice.loc[1,'longitude']
distance(lat1, lon2, lat2, lon2)

### Exercises

Question 1: How many vehicles are there?

Question 2: How many unique routes are there?

### Joining dataframes
- Use an inner join in case some vehicles are missing at certain times

In [ ]:
df0 = pd.read_csv('vehicles_0.csv', index_col=0).set_index('vehicleId')
df1 = pd.read_csv('vehicles_1.csv', index_col=0).set_index('vehicleId')
df0 = df0[['latitude', 'longitude']]
df1 = df1[['latitude', 'longitude']]

df_joined = df0.join(df1, how='inner', rsuffix='1')
df_joined.head()

### Use `apply` on the joined DataFrame to apply the distance function to each row

In [ ]:
from math import cos, asin, sqrt
def distance(lat1, lon1, lat2, lon2):
    p = 0.017453292519943295
    a = 0.5 - cos((lat2 - lat1) * p)/2 + cos(lat1 * p) * cos(lat2 * p) * (1 - cos((lon2 - lon1) * p)) / 2
    return 12742 * asin(sqrt(a))


def distance_row(row):
    # Unpack row and pass it to `distance`    
    
    # Method 1:
    # lat1, lon1, lat2, lon2 = row
    # return distance(lat1, lon1, lat2, lon2)
    
    # Method 2:
    # Use the * operator to unpack row into variables
    
    return distance(*row)

In [ ]:
df_joined.apply(distance_row, axis=1)